<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных-и-знакомство-с-ними" data-toc-modified-id="Загрузка-данных-и-знакомство-с-ними-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных и знакомство с ними</a></span></li><li><span><a href="#Количество-выпущенных-книг-после-01.01.2000г." data-toc-modified-id="Количество-выпущенных-книг-после-01.01.2000г.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Количество выпущенных книг после 01.01.2000г.</a></span></li><li><span><a href="#Количество-обзоров-и-средняя-оценка-для-каждой-книги" data-toc-modified-id="Количество-обзоров-и-средняя-оценка-для-каждой-книги-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Количество обзоров и средняя оценка для каждой книги</a></span></li><li><span><a href="#Издательство,-выпустившее-наибольшее-количество-книг" data-toc-modified-id="Издательство,-выпустившее-наибольшее-количество-книг-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Издательство, выпустившее наибольшее количество книг</a></span></li><li><span><a href="#Автор-с-самой-высокой-средней-оценкой-книг" data-toc-modified-id="Автор-с-самой-высокой-средней-оценкой-книг-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Автор с самой высокой средней оценкой книг</a></span></li><li><span><a href="#Среднее-количество-обзоров-активных-пользователей" data-toc-modified-id="Среднее-количество-обзоров-активных-пользователей-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Среднее количество обзоров активных пользователей</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Анализ базы данных сервиса для чтения книг

В нашем распоряжении доступ к базе данных сервиса для чтения кинг с информацией о книгах, издательствах, авторах и пользовательских обзорах. На основании анализа этих данных мы сформулируем предложение для нового продукта. Для этого оценим обзоры и оценки по книгам, определим самое продуктивное издательство и самого популярного автора.

## Загрузка данных и знакомство с ними

Для начала импортируем необходимые нам библиотеки и подключимся к базе:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', 
             'pwd': 'Sdf4$2;d-d30pp', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Теперь ознакомимся с содержимым всех пяти таблиц:

In [2]:
query = ''' SELECT *
FROM books
LIMIT 5'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' SELECT *
FROM authors
LIMIT 5'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' SELECT *
FROM ratings
LIMIT 5'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' SELECT *
FROM reviews
LIMIT 5'''
display(pd.io.sql.read_sql(query, con = engine))

query = ''' SELECT *
FROM publishers
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Количество выпущенных книг после 01.01.2000г.

Посчитаем, сколько книг вышло, начиная с 1 января 2000 года:

In [3]:
query = ''' SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01' '''
pd.io.sql.read_sql(query, con = engine)

,count
0,821


Итак, **с 2000-го года была выпущена 821 книга**. 

## Количество обзоров и средняя оценка для каждой книги

Далее посчитаем для каждой книги количество обзоров и среднюю оценку:

In [4]:
query = ''' 
SELECT b.title, 
        COUNT(DISTINCT rev.review_id) count_of_reviews, 
        ROUND(AVG(rat.rating), 1) average_rating
FROM books AS b
LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
LEFT JOIN ratings AS rat ON rev.book_id=rat.book_id
GROUP BY b.book_id
ORDER BY count_of_reviews DESC,
        average_rating DESC
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,title,count_of_reviews,average_rating
0,Twilight (Twilight #1),7,3.7
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.3
3,The Book Thief,6,4.3
4,The Glass Castle,6,4.2


**Самое большое количество обзоров и высоких средних оценок у книг "Сумерки", "Книжный вор", а также у второй и третьей части "Гарри Поттера".**

## Издательство, выпустившее наибольшее количество книг

Определим издательство, которое выпустило наибольшее число книг. Возьмем в анализ только публикации толще 50 страниц.

In [5]:
query = ''' SELECT pub.publisher
FROM publishers AS pub
LEFT JOIN books AS b ON pub.publisher_id=b.publisher_id
WHERE b.num_pages > 50
GROUP BY pub.publisher_id
ORDER BY COUNT(b.book_id) DESC
LIMIT 1'''
pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


**Самое продуктивное издательство - "Penguin Books".**

## Автор с самой высокой средней оценкой книг

Определим автора с самой высокой средней оценкой книг. Учитывать будем только книги с 50 и более оценками.

In [9]:
query = ''' 
SELECT a.author, 
        ROUND(AVG(rat.rating), 1) average_rating
FROM books AS b
LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
LEFT JOIN authors AS a ON b.author_id=a.author_id
WHERE b.book_id IN (SELECT b.book_id
FROM books AS b
LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
GROUP BY b.book_id
HAVING COUNT(rat.rating) >= 50)
GROUP BY a.author
ORDER BY ROUND(AVG(rat.rating), 1) DESC
'''
pd.io.sql.read_sql(query, con = engine)

,author,average_rating
0,Markus Zusak/Cao Xuân Việt Khương,4.3
1,J.K. Rowling/Mary GrandPré,4.3
2,J.R.R. Tolkien,4.2
3,Louisa May Alcott,4.2
4,Rick Riordan,4.1
5,William Golding,3.9
6,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.8
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.8
8,Lois Lowry,3.8
9,J.D. Salinger,3.8


В топе авторов с книгами с самыми высокими средними оценками: **Джоан Роулинг и Маркус Зузак**.

## Среднее количество обзоров активных пользователей

Посчитаем среднее количество обзоров от пользователей, которые поставили более 50 оценок:

In [8]:
query = ''' SELECT ROUND(AVG(mean_count.count_review)) AS mean_count_of_reviews
FROM (SELECT username, COUNT(review_id) AS count_review
FROM reviews
WHERE username IN (SELECT username
FROM ratings 
GROUP BY username
HAVING COUNT(rating_id) > 50)
GROUP BY username) AS mean_count
'''
pd.io.sql.read_sql(query, con = engine)

,mean_count_of_reviews
0,24.0


**Таким образом, активные пользователи написали, в среднем, по 24 обзора.**

## Вывод

По итогам анализа мы имеем следующее:
- с 2000-го года была выпущена 821 книга;
- самое большое количество обзоров и высоких средних оценок у книг "Сумерки", "Книжный вор", а также у второй и третьей части "Гарри Поттера";
- в топе авторов с книгами с самыми высокими средними оценками: Джоан Роулинг и Маркус Зузак;
- самое продуктивное издательство - "Penguin Books";
- активные пользователи написали, в среднем, по 24 обзора.

Если мы посмотрим на самые высокооценённые позиции, то обратим внимание, что эти книги успешно экранизированы: "Сумерки", "Гарри Поттер" - это очевидно; произведение "Книжный вор" Маркуса Зузака было экранизировано под названием "Воровка книг". 

Вероятно, стоит делать расчёт на успешно экранизированные книги, так как после просмотра зрителям часто хочется обратиться к первоисточнику истории.